In [1]:
TEXT_DATA_DIR = '/home/ncarkaci/workspace/tr-preprocessing/dataset/'

In [ ]:
import os
import preprocessing



In [ ]:
filename = preprocessing.mergeFiles(TEXT_DATA_DIR)

In [ ]:
with open (filename, 'r') as input_file:
    text = input_file.read()

In [ ]:
text = preprocessing.replaceChars(text)
text = preprocessing.toLowercase(text)
text = preprocessing.removeUndesiredCharsFromText(text,False, False, False)

In [ ]:
text = text.replace('\n', ' ')
text = text.replace('  ', ' ')

In [ ]:
text[:1000]

In [ ]:
corpus = text.split()

In [ ]:
len(corpus)

In [ ]:
vocabulary = set(corpus)

In [ ]:
len(vocabulary)

In [ ]:
vocab = list(vocabulary)

In [ ]:
vocab[:100]

In [ ]:
from collections import Counter

In [ ]:
col = Counter(corpus)

In [ ]:
col.most_common

In [ ]:
import os
import preprocessing
sentences = []  # list of text samples

for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    
    with open(path, 'rt') as input_file:

            text = input_file.read()
            text = preprocessing.replaceChars(text)
            text = preprocessing.toLowercase(text)
            text = preprocessing.removeUndesiredCharsFromText(text,False, False, False)
            sentences.append(text.split())

print('Found %s texts.' % len(sentences))

In [ ]:
import gensim

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# train word2vec on the two sentences
word_model = gensim.models.word2vec.Word2Vec(corpus, size=num_features, window=context, min_count=min_word_count, workers=num_workers)

In [ ]:
model_name = str(num_features)+'_features_'+str(min_word_count)+'_minwords_'+str(context)+'_context.word2vec'
word_model.save(model_name) # save word2vec model into file

In [ ]:
word_model.wv.vocab.keys()

In [ ]:
len(word_model.wv['akp'])  # numpy vector of a word

In [ ]:
word_model.most_similar('istanbul') # return similar

In [ ]:
word_model.wv.most_similar(positive=['tayyip', 'akp'], negative=['chp'])

In [ ]:
word_model.similarity('tayyip','akp')

In [ ]:
word_model.wv.syn0.shape[0]

In [ ]:
vocabulary_size = word_model.wv.syn0.shape[0]

In [ ]:
word_model.wv.syn0.shape[1]

In [ ]:
# Code tried to prepare LSTM model for word generation
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout


input_dimention = word_model.wv.syn0.shape[0]
output_dimention = word_model.wv.syn0.shape[1]


embedding_layer = Embedding(input_dim=input_dimention, output_dim=output_dimention, weights=[ word_model.wv.syn0])


In [ ]:
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop

# define the LSTM model
print ('define the LSTM model')

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(output_dimention, return_sequences=True))
model.add(TimeDistributed(Dense(input_dimention)))   
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))


'''
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(256, input_shape=(input_dimention, output_dimention), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(input_dimention, activation='softmax'))
'''

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
# define the checkpoint
print ('define the checkpoint')
filepath="word2vec-text-generation-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
sequence_length = 100

X = np.zeros((int(len(data)/sequence_length), sequence_length, vocabulary_size))
y = np.zeros((int(len(data)/sequence_length), sequence_length, vocabulary_size))

In [ ]:
for i in range(0, int(len(data)/sequence_length)):
    X_sequence     = data[i*sequence_length:(i+1)*sequence_length]
    X_sequence_ix  = [char_to_index_dictionary[value] for value in X_sequence]
    input_sequence = np.zeros((sequence_length, vocabulary_size))
    for j in range(sequence_length):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence
    
    y_sequence      = data[i*sequence_length+1:(i+1)*sequence_length+1]
    y_sequence_ix   = [char_to_index_dictionary[value] for value in y_sequence]
    target_sequence = np.zeros((sequence_length,vocabulary_size))
    for j in range(sequence_length):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i]= target_sequence

In [ ]:
# fit the model
print('Fit model')
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)